Write Auto-Reset:


In [ ]:
# gymnax/environments/environment.py

@functools.partial(jax.jit, static_argnums=(0,))
def step(
    self,
    key: chex.PRNGKey,
    state: TEnvState,
    action: Union[int, float, chex.Array],
    params: Optional[TEnvParams] = None,
) -> Tuple[chex.Array, TEnvState, jnp.ndarray, jnp.ndarray, Dict[Any, Any]]:
    """Performs step transitions in the environment."""
    # Use default env parameters if no others specified
    if params is None:
        params = self.default_params
    key, key_reset = jax.random.split(key)
    obs_st, state_st, reward, done, info = self.step_env(key, state, action, params)
    obs_re, state_re = self.reset_env(key_reset, params)
    # Auto-reset environment based on termination
    
    # The `state` is recursively updated to be the reset state if done is True
    state = jax.tree_map(
        lambda x, y: jax.lax.select(done, x, y), state_re, state_st
    )
    obs = jax.lax.select(done, obs_re, obs_st)
    return obs, state, reward, done, info